In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab
    
    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/bikes.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M3/CLASE_16_Regresion_Lineal_Multiple/Data/bikes.csv?token=AA4GFHIG665I3BPVQCFY3US63APZM
    
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

[<img src="https://www.digitalhouse.com/ar/logo-DH.png" width="400" height="200" align='right'>](http://digitalhouse.com.ar/)

# PRACTICA GUIADA: Regresión lineal con predictores cualitativos

# 1 **Introducción**
1.Introduccón al dataset "bikeshare" 
* Leyendo los datos
* Visualizando los datos

2.Regresión lineal
* Forma de regresión lineal
* Construir un modelo de regresión lineal
* Uso del modelo para la predicción
* ¿Es importante la escala de las features?

3.Trabajando con múltiples features
* Visualizando los datos 
* Agregando más features al modelo

4.Eligiendo entre modelos
* Selección de features
* Métricas de evaluación para problemas de regresión
* Comparación de modelos con sets de entrenamiento/test y RMSE
* Comparando RMSE de prueba con RMSE nulo (baseline)
       
5.Creando features
* Manejo de feactures categóricas
* Ingeniería de features



**===========================================================================================**

### 1.1 Importando datos

Vamos a trabajar con un conjunto de datos sobre alquileres de bicicletas que fue utilizado en un concurso de Kaggle.


Se proporcionan datos de alquiler por hora que abarcan dos años. El conjunto de entrenamiento se compone de los primeros 19 días de cada mes, mientras que el conjunto de test es del día 20 al final del mes.

**_Queremos predecir el número total de bicicletas alquiladas durante cada hora cubierta por el conjunto de test, utilizando sólo la información disponible en el set de entrenamiento._**


**CAMPOS DEL DATASET**

**datetime** - dia y hora - timestamp

**season** -  1 = primavera, 2 = verano, 3 = otoño, 4 = invierno

**holiday** - feriado

**workingday** - día de la semana

**weather** - 

              1: Clear, Few clouds, Partly cloudy, Partly cloudy
              2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
              3: Light Snow, Light Rain + Thunderstorm + Scattered clouds,
                 Light Rain + Scattered clouds
              4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 

**temp** - temperatura en Celsius

**atemp** - sensación térmica

**humidity** - humedad

**windspeed** - velocidad del viento

**casual** - ususarios no-registrados (number of non-registered user rentals initiated)

**registered** - ususarios registrados (number of registered user rentals initiated)

**count** - total de alquileres


In [ ]:
# Leemos los datos y seteamos el datetime como índice.
import pandas as pd

bikes = pd.read_csv('../Data/bikes.csv', index_col='datetime', parse_dates=True)

In [ ]:
type(bikes)

In [ ]:
bikes.shape

In [ ]:
bikes.sample(10)

**Preguntas:**

*     ¿Qué representa cada observación?
*     ¿Cuál es la variable de respuesta?
*     ¿Cuántas variables hay?

In [ ]:
# Dado que "count" es un método de pandas, es conviente renombrar la columna:

bikes.rename(columns={'count':'total'}, inplace=True)

In [ ]:
bikes.sample(2)

### 1.2 Visualización

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Definimos parámetros globales para matplotlib.
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 16

In [ ]:
# Ploteamos puntos con Pandas
bikes.plot(kind='scatter', x='temp', y='total', alpha=0.2);

In [ ]:
# Ajuste con Seaborn (modelo lineal) 
sns.lmplot(x='temp', y='total', data=bikes, aspect=1.45,\
                                scatter_kws={'alpha':0.2});

# 2 Regresión Lineal

## 2.1 Repaso: forma del modelo lineal

![Ec. Recta](http://askingroom.com/blog/wp-content/uploads/2018/11/Ecuaci%C3%B3n-de-la-Recta-1-300x143.png)

$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + ... + \beta_nx_n$
- $y$ es la variable dependiente (es la respuesta)
- $\beta_0$ es el intercepto
- $\beta_1$ es el coeficiente para $x_1$ 
- $\beta_n$ es el coeficiente para $x_n$

Los **$\beta$** son los llamados **_Coeficientes del modelo_**

- Estos valores son estimados (o "aprendidos") durante el proceso de adaptación del modelo usando el criterio ** mínimos cuadrados **.
- Específicamente, encontramos la línea (matemáticamente) que minimiza la suma ** de cuadrados de residuos ** (o "suma de errores cuadráticos").
- Y una vez que hemos aprendido estos coeficientes, podemos usar el modelo para predecir la respuesta.


![Regresión Lineal](https://cdn-images-1.medium.com/max/720/1*yLeh6JjWHenfH4zFOA3HpQ.png)

*https://medium.com/@337_73413/machine-learning-a-micro-primer-with-a-lawyers-perspective-cfe5a69c114d*

## 2.2 Construyendo el modelo de Regresión Lineal

Construir un modelo en **[Scikit-Learn](https://scikit-learn.org/stable/)** lleva 5 pasos.

1.   Preparar los datos en una **matriz de features y un array target**.

2.   Elegir una clase de modelo importando la **clase de estimador** apropiado de Scikit-Learn.

3.   Seleccionar los **hiperparámetros** del modelo **instanciando** la clase con los valores deseados

4.   Ajustar el modelo a los datos invocando el método** fit()** de la instancia del modelo.

5.   Aplicar el modelo a **nuevos datos**:


Empezamos por una regresión lineal simple!

In [ ]:
# Creamos X e y

feature_cols = ['temp']
X = bikes[feature_cols]
y = bikes.total

In [ ]:
type(y)

In [ ]:
X.shape

In [ ]:
# Importamos paquete, instanciamos el estimador y fiteamos el modelo ("classic" sklearn!)

from sklearn.linear_model import LinearRegression

In [ ]:
# Instanciamos el modelo

linreg = LinearRegression()

In [ ]:
# Entrenamos el modelo 

linreg.fit(X, y)

In [ ]:
# Imprimimos coeficientes

print (linreg.intercept_)
print (linreg.coef_)

Interpretación del  **intercepto ** ($\beta_0$):

- Es el valor esperado de $ y $ cuando $ x $ = 0.
- Por lo tanto, es el número esperado de alquileres cuando la temperatura es de 0 grados Celsius.
- ** Nota: ** No siempre tiene sentido interpretar el intercepto. (¿Por qué?)

Interpretación del coeficiente de ** "temp" ** ($\beta_1$):

- Es el cambio en $ y $ dividido por cambio en $ x $, o la "pendiente".
- Así, un aumento de la temperatura de 1 grado Celsius está ** asociado con ** un aumento de alquiler de 9.17 bicicletas.
 $ \beta_1 $ sería ** negativo ** si un aumento en la temperatura se asociara con una ** disminución ** en los alquileres.

## 2.3 Usando el modelo para predecir

¿Cuántos alquileres de bicicletas podríamos predecir si la temperatura era de 25 grados Celsius?

In [ ]:
# Aplicando la fórmula manualmente

test = 25

linreg.intercept_ + linreg.coef_*test

In [ ]:
# usando el método del objeto
import numpy as np

test_sklearn = np.array(test).reshape(-1,1)

linreg.predict(test_sklearn)

## 2.4 ¿Es importante la escala de las features?

Digamos que la temperatura se midió en grados Fahrenheit, en lugar de Celsius. ¿Cómo afecta esto al modelo?

In [ ]:
# Creamos una nueva columna para la temperatura en Fahrenheit
# ℃ = (℉ - 32)/1.8

bikes['temp_F'] = bikes.temp * 1.8 + 32
bikes.head()

In [ ]:
# Nuevamente ajustamos con Seaborn 
sns.lmplot(x='temp_F', y='total', data=bikes, aspect=1.45, scatter_kws={'alpha':0.2});

In [ ]:
# creamos nuevamente X e y

feature_cols = ['temp_F']
X = bikes[feature_cols]
y = bikes.total

# Instanciamos el modelo y fiteamos

linreg = LinearRegression()
linreg.fit(X, y)

# Imprimimos coeficientes

print (linreg.intercept_)
print (linreg.coef_)

In [ ]:
# Convertimos 25°c a °F

test_en_f = 25 * 1.8 + 32

In [ ]:
# Predicción de alquileres para 77°F

test_sklearn_en_f = np.array(test_en_f).reshape(-1,1)

linreg.predict(test_sklearn_en_f)

** Conclusión: ** La escala de las características es ** irrelevante ** para los modelos de regresión lineal. Al cambiar la escala, simplemente cambiamos nuestra ** interpretación ** de los coeficientes.

In [ ]:
# borramos la columna temp_F 

bikes.drop('temp_F', axis=1, inplace=True)

## 2.5 Visualización de los datos 2

In [ ]:
# Exploramos más features

feature_cols = ['temp', 'season', 'weather', 'humidity']

In [ ]:
# plots múltiples en seaborn

sns.pairplot(bikes, x_vars=feature_cols, y_vars='total', kind='reg',\
                                                height=5, aspect=1);

In [ ]:
# matriz de correlación (rangos de 1 a -1)

bikes.corr()

In [ ]:
# visualizamos la matriz de correlación en Seaborn usando a heatmap

sns.heatmap(bikes.corr(), vmin=-1, vmax=1, center=0, cmap="YlGnBu");

<center>__¿Qué relaciones se observan?__

## 2.6 Sumando más features

In [ ]:
# creamos lista de features

feature_cols = ['temp', 'humidity']

In [ ]:
# creamos nuevamente X and y
X = bikes[feature_cols]
y = bikes.total

# creamos el modelo y fiteamos
linreg = LinearRegression()
linreg.fit(X, y)

# Imprimimos coeficientes
print (linreg.intercept_)
print (linreg.coef_)

In [ ]:
## para observarlo mejor miramos el nombre con el coeficiente
list(zip(feature_cols, linreg.coef_))

# 3 Eligiendo entre modelos

## 3.1 Selección de features

¿Cómo elegimos cuales features incluir en el modelo? Vamos a usar los sets de   **entrenamiento** y **test** (y eventualmente **validación cruzada**).


## 3.2 Métricas de evaluación para problemas de regresión

 Hay tres métricas de evaluación comunes para problemas de regresión:

** El error absoluto medio ** (MAE) es la media del valor absoluto de los errores:

$$ \frac 1n\sum_ {i = 1}^n |y_i-\hat{y}_i| $$

** Mean Squared Error ** (MSE) es la media de los errores al cuadrado:

$$ \frac 1n\sum_ {i = 1}^n(y_i- \hat{y}_i)^2 $$

** Error cuadrático medio raíz ** (RMSE) es la raíz cuadrada de la media de los errores al cuadrado:

$$ \sqrt{\frac 1n\sum_{i = 1}^n(y_i- \hat{y}_i)^2} $$

In [ ]:
# Ejemplo de valores de respuesta verdaderos y predichos
true = [10, 7, 5, 5, 10, 8, 8, 15, 12]
pred = [12, 8, 3, 4, 12, 9, 8, 12, 13]

In [ ]:
# ¿Calculamos métricas a mano? ¡Por supuesto que no!

from sklearn import metrics
import numpy as np
print ('MAE:', metrics.mean_absolute_error(true, pred))
print ('MSE:', metrics.mean_squared_error(true, pred))
print ('RMSE:', np.sqrt(metrics.mean_squared_error(true, pred)))
print ('R2:', metrics.r2_score(true, pred))

Comparando estas métricas:

- ** MAE **  es el error promedio.
- ** MSE **  "penaliza" errores grandes.
- ** RMSE **  es interpretable, tiene las mismas unidades  que la "y".
- ** $R^2$ ** es la proporción de la varianza total de $Y$ explicada por el modelo

Con excepción de R2, todas estas son ** funciones de pérdida **, porque queremos minimizarlas.

Ejemplo adicional, para ver cómo MSE / RMSE penalizan más a los errores más grandes:

In [ ]:
# con los mismos valores de antes para true
true = [10, 7, 5, 5, 10, 8, 8, 15, 12]

# nuevo set de valores para la predicción
pred = [12, 8, 3, 4, 12, 9, 8, 22, 13]

# MAE se incrementa levemente
print ('MAE:', metrics.mean_absolute_error(true, pred))

# MSE y RMSE son más grandes que antes. A su vez, R2 empeora su performance.
print ('MSE:', metrics.mean_squared_error(true, pred))
print ('RMSE:', np.sqrt(metrics.mean_squared_error(true, pred)))
print ('R2:', metrics.r2_score(true, pred))

## 3.3 Comparando modelos usando sets de entrenamiento/test y RMSE

### Train-Test Split

![Train-Test Split](https://cdn-images-1.medium.com/max/720/1*3g5RtdlP85EUsF-peOA1-g.png)

*https://medium.com/@hi.martinez/train-test-split-cross-validation-you-b87f662445e1*

In [ ]:
# Definimos una función que acepta una lista de features, hace el split entre train y test,
# reservando un 25% de las observaciones para testeo, y devuelve la prueba RMSE.

from sklearn.model_selection import train_test_split

def train_test_rmse(feature_cols):
    X = bikes[feature_cols]
    y = bikes.total
    # Como estamos trabajando con observaciones ordenadas en el tiempo, ponemos
    # shuffle=False para evitar data leakage
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    return np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [ ]:
# comparamos diferentes ensambles de features
print (train_test_rmse(['temp', 'humidity']))
print (train_test_rmse(['temp', 'humidity','windspeed']))

In [ ]:
# Usando features no permitidas!!!
# Son las features que componen la variable de respuesta.
# Por definición la cantidad total de bicis prestadas es la suma de bicis prestadas a usuarios casuales y registrados
# No tiene ningún sentido hacer un modelo de esto
# Pero como la relación es tan directa, las métricas de error son bajísimas.
print (train_test_rmse(['casual', 'registered']))

In [ ]:
# Chequeamos que las columnas son perfectamente dependientes.
np.all(bikes.casual + bikes.registered == bikes.total)

# 4 Creando Features

## 4.1 Manejo de variables categóricas

Scikit-learn espera que todos los features sean numéricos. Entonces, ¿cómo incluimos una característica categórica en nuestro modelo?

- ** Categorías ordenadas: ** transformarlas en valores numéricos que representan ese orden 
- ** Categorías no ordenadas: ** utilizar codificación ficticia (0/1) (Variables-dummy)

¿Cuáles son las características categóricas de nuestro conjunto de datos?

- ** Categorías ordenadas: ** weather (ya codificado con valores numéricos que representan el orden)
- ** Categorías no ordenadas: ** season (necesita variables dummy), holiday (Ya está codificada como dummy), workingday (ya está codificada como dummy)

Para la estación, no podemos simplemente dejar la codificación como 1 = primavera, 2 = verano, 3 = otoño y 4 = invierno, porque eso implicaría una ** relación ordenada **. En cambio, creamos ** variables dummies: **

In [ ]:
# crear variables dummies
season_dummies = pd.get_dummies(bikes.season, prefix='season')

# imprimimos para ver 5 filas cualquieras
season_dummies.sample(n=5, random_state=1)

Nota: El método `get_dummies` puede recibir un dataframe entero, en ese caso no modifica las variables numéricas y genera dummies para todas las categóricas que encuentre. 

Sin embargo, en realidad sólo necesitamos ** tres variables dummy (no cuatro) **, y por lo tanto vamos a dropear la primera variable dummy. 
¿Por qué?

In [ ]:
# Salteamos la primer columna
season_dummies.drop(season_dummies.columns[0], axis=1, inplace=True)

# imprimimos 5 filas cualquieras
season_dummies.sample(n=5, random_state=1)

En general, si se tiene una feature categórica con ** k valores posibles **, se tienen que crear ** k-1 variables dummies **.

In [ ]:
# Concatenar el DataFrame original y el dummy DataFrame (axis = 0 significa filas, axis = 1 significa columnas)
bikes = pd.concat([bikes, season_dummies], axis=1)

# imprimimos 5 filas cualquieras
bikes.sample(n=5)

In [ ]:
# Incluímos variables dummies

feature_cols = ['temp', 'season_2', 'season_3', 'season_4', 'humidity']
X = bikes[feature_cols]
y = bikes.total
linreg = LinearRegression()
linreg.fit(X, y)


list(zip(feature_cols, linreg.coef_))

¿Cómo interpretamos los coeficientes de season? ** se miden con respecto a la línea de base (spring) **:

- Manteniendo todas las demás características fijas, ** summer ** se asocia con una ** disminución de alquiler de 3.39 bicicletas ** en comparación con spring.
- Manteniendo todas las demás características fijas, ** fall ** se asocia con una ** disminución de alquiler de 41,73 bicicletas ** en comparación con spring.
- Manteniendo todas las demás características fijas, ** winter ** se asocia con un ** aumento de alquiler de 64,4 bicicletas ** en comparación con spring.

¿Qué pasa si cambiamos la dummy que se definió como la línea de base? ¿Cambiarían los efectos?

- No, simplemente cambiaría nuestra ** interpretación ** de los coeficientes.

** Importante: ** La codificación por dummies es relevante para todos los modelos de aprendizaje automático, no sólo para los modelos de regresión lineal.

In [ ]:
print (train_test_rmse(['temp', 'season_2', 'season_3', 'season_4', 'humidity']))

## 4.2 Conclusiones: Comparación de la regresión lineal con otros modelos

Ventajas de la regresión lineal:

- Simple de explicar
- Muy interpretable
- El entrenamiento y predicción de modelos son rápidos
- Es invariante a cambios en la escala de los features. 
- No se puede estimar por mínimos cuadrados si el número de features es mayor al número de observaciones. 


Desventajas de la regresión lineal:

- El rendimiento es (generalmente) no competitivo con los mejores métodos de aprendizaje supervisado debido a un alto sesgo

# BONUS

## Cross-Validation

![Flujo Cross-Validation](https://cdn-images-1.medium.com/max/720/1*pJ5jQHPfHDyuJa4-7LR11Q.png)

*https://medium.com/@hi.martinez/train-test-split-cross-validation-you-b87f662445e1*